In [85]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import bql
from pandas.tseries.offsets import BDay
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar


today = pd.datetime.today()
startrange = today - BDay(2)
endrange = today - BDay(1)
startrange = startrange.to_pydatetime()
endrange = endrange.to_pydatetime()
cal = calendar()
holidays = cal.holidays(start=startrange.strftime('%Y-%m-%d'), end=endrange.strftime('%Y-%m-%d'))
#startrange = startrange.strftime('%Y-%m-%d')
#endrange = endrange.strftime('%Y-%m-%d')
if startrange.strftime('%Y-%m-%d') in holidays:
    startrange = (startrange-BDay(1)).strftime('%Y-%m-%d')
    endrange = endrange.strftime('%Y-%m-%d')
elif endrange.strftime('%Y-%m-%d') in holidays:
    startrange = (startrange-BDay(1)).strftime('%Y-%m-%d')
    endrange = (endrange-BDay(1)).strftime('%Y-%m-%d')
else:
    startrange = startrange.strftime('%Y-%m-%d')
    endrange = endrange.strftime('%Y-%m-%d')

In [ ]:
bq = bql.Service()


CUSIPDB = pd.read_excel('CUSIP Database Sectors.xlsx')


sec_name = bq.data.SECURITY_DES()
MTG_CMO_CLASS = bq.data.MTG_CMO_CLASS()
price = bq.data.px_last(start=startrange,end=endrange,per='D')


sec_name_response = bq.execute(bql.Request(CUSIPDB['Identifier'][0:5000,],sec_name))
sec_name_response2 = bq.execute(bql.Request(CUSIPDB['Identifier'][5000:len(CUSIPDB['Identifier']),],sec_name))
MTG_CMO_CLASSresponse = bq.execute(bql.Request(CUSIPDB['Identifier'][0:5000,],MTG_CMO_CLASS))
MTG_CMO_CLASSresponse2 = bq.execute(bql.Request(CUSIPDB['Identifier'][5000:len(CUSIPDB['Identifier']),],MTG_CMO_CLASS))
priceresponse = bq.execute(bql.Request(CUSIPDB['Identifier'][0:5000,],price))
priceresponse2 = bq.execute(bql.Request(CUSIPDB['Identifier'][5000:5152,],price))



In [ ]:
price = bq.data.px_last(start=startrange,end=endrange,per='D')

pricedf = priceresponse[0].df().drop('CURRENCY',axis=1).reset_index().set_index('DATE').groupby('DATE').apply(lambda g: g.set_index('ID').unstack()).T
pricedf = pricedf.reset_index().drop('level_0',axis=1).set_index('ID')
pricedf = pricedf.dropna(axis='columns',how='all')

In [ ]:
pricedftwo = priceresponse2[0].df().drop('CURRENCY',axis=1).reset_index().set_index('DATE').groupby('DATE').apply(lambda g: g.set_index('ID').unstack()).T
pricedftwo = pricedftwo.reset_index().drop('level_0',axis=1).set_index('ID')
pricedftwo = pricedftwo.dropna(axis='columns',how='all')

In [ ]:
masterprice = pd.concat([pricedf,pricedftwo])

In [ ]:
MasterDF = pd.concat([sec_name_response[0].df(),sec_name_response2[0].df()])
MasterDF = pd.concat([MasterDF,pd.concat([MTG_CMO_CLASSresponse[0].df(),MTG_CMO_CLASSresponse2[0].df()])],axis=1)
MasterDF.columns = ['Sec_Desc','Class']
MasterDF = pd.concat([MasterDF,CUSIPDB.set_index('Identifier')],axis=1)
MasterDF = pd.concat([MasterDF,masterprice],axis=1,sort=False)

In [ ]:
MasterDF.iloc[:,[3]] = np.where(MasterDF.iloc[:,[3]].isnull(),"Paid Off",MasterDF.iloc[:,[3]])
MasterDF.iloc[:,[4]] = np.where(MasterDF.iloc[:,[4]].isnull(),"Paid Off",MasterDF.iloc[:,[4]])

filtereddf = MasterDF[MasterDF[MasterDF.columns[4]]!= "Paid Off"]
filtereddf = filtereddf[filtereddf[filtereddf.columns[3]]!= "Paid Off"]
filtereddf[filtereddf.columns[4]] = filtereddf[filtereddf.columns[4]].astype('float')
filtereddf[filtereddf.columns[3]] = filtereddf[filtereddf.columns[3]].astype('float')
filtereddf['DoD Change'] = filtereddf[filtereddf.columns[4]]-filtereddf[filtereddf.columns[3]]
filtereddf['Abs Change'] = abs(filtereddf['DoD Change'])
filtereddf['Abs Change'] = filtereddf['Abs Change'].astype('float')
filtereddf['Abs % Change']= abs(((filtereddf[filtereddf.columns[4]]-filtereddf[filtereddf.columns[3]])/filtereddf[filtereddf.columns[3]])*100)
filtereddf = filtereddf.round(2)
filtereddf.columns.values[3] = filtereddf.columns[3].strftime('%m/%d/%Y')
filtereddf.columns.values[4] = filtereddf.columns[4].strftime('%m/%d/%Y')

In [ ]:
sectorfilts = list(set(filtereddf['Sector']))
sec_dfs = []
for i in range(len(sectorfilts)):
    sec_dfs.append(filtereddf.groupby('Sector').get_group(sectorfilts[i]).copy())

top10movers = filtereddf.nlargest(10,'Abs Change')
top10movers = top10movers.sort_values(by=['Abs Change'],ascending = False)
toppercentmovers = filtereddf[filtereddf['Abs % Change']>=1]
toppercentmovers = toppercentmovers.sort_values(by='Abs % Change',ascending = False)
   
    
def tablefilterer(x):
    t10 = x.nlargest(10,'Abs Change')
    t10 = t10.sort_values(by=['Abs Change'],ascending = False)
    return t10;

toptenmoverslist = []
for i in range(len(sec_dfs)):
    toptenmoverslist.append(tablefilterer(sec_dfs[i]).sort_values(by=['Class']))

# Top Ten Movers Universe

In [ ]:
top10movers

# Sector 1

In [ ]:
toptenmoverslist[0]

# Sector 2

In [ ]:
toptenmoverslist[1]

# Sector 3

In [ ]:
toptenmoverslist[2]

# Sector 4

In [ ]:
toptenmoverslist[3]

# Sector 5

In [ ]:
toptenmoverslist[4]

# Sector 6

In [ ]:
toptenmoverslist[5]

# Universe Part 1 >1%

In [ ]:
toppercentmovers[0:(len(toppercentmovers)-30)]

# Universe Part 2 >1%

In [ ]:
toppercentmovers[(len(toppercentmovers)-30):len(toppercentmovers)]